In [2]:
from mysql.connector import connect
from elasticsearch import  helpers
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
sys.path.append(r'D:\markding_git\big-data-ai-integration-platform\airflow\dags\code')
import es_mapping
from connection import ElasticSearchConnectionManager


# Alphabet generator

In [ ]:
# use the string module
import string
alphabet_list = list(string.ascii_lowercase)

print(alphabet_list)

# ES connection settup

In [ ]:
# def create_es_connection():
#     # Load certificate path and credentials from environment variables
#     cafile_path = os.getenv('ES_CAFILE_PATH', r"D:\markding_git\big-data-ai-integration-platform\search_system\es_certs\http_ca.crt")
#     es_host = os.getenv('ES_HOST', 'http://localhost:9200')
#     es_user = os.getenv('ES_USER', 'elastic')
#     es_password = os.getenv('ES_PASSWORD', 'gAcstb8v-lFCVzCBC__a')
#     # uWzW*VOzBZknR-Jt3T7i
#     context = create_default_context(cafile=cafile_path)
#     # context.check_hostname = True  # Ensure hostname is checked
#     # context.verify_mode = CERT_REQUIRED  # Ensure the certificate is verified

#     context.check_hostname = False
#     context.verify_mode = CERT_NONE
    
#     es = Elasticsearch(
#                 [es_host],
#                 http_auth=(es_user, es_password),
#                 verify_certs=False,
#                 )
#     return es

# es = create_es_connection()
es = ElasticSearchConnectionManager.get_instance()

print(es.info())

es_health = es.health_report()
print(es_health['status'])
indicators = es_health['indicators']
[f"status_of_[{i}] : {indicators[i]['status']}" for i in indicators.keys()]

In [ ]:
def es_mapping(df, column_types_dict):
    """
    Generate Elasticsearch mapping based on the DataFrame and a column-to-Python type dictionary.

    Args:
        df (pandas.DataFrame): The DataFrame to be indexed.
        column_types_dict (dict): Dictionary mapping columns to Python types.

    Returns:
        dict: Elasticsearch mapping dictionary.
    """

    # Define a mapping from Python types to Elasticsearch types
    type_conversion = {
        'object': 'keyword',  # Typically strings in Pandas
        'int64': 'integer',
        'int32': 'integer',
        'float64': 'float',
        'float32': 'float',
        'bool': 'boolean',
        'datetime64[ns]': 'date',
        'timedelta[ns]': 'long',
        'category': 'keyword',
    }

    # Generate the Elasticsearch mapping using the provided column types
    mapping = {
        "mappings": {
            "properties": {
                col: {
                    "type": type_conversion.get(column_types_dict[col], "text"),  # Default to "text"
                    "ignore_malformed": True if type_conversion.get(column_types_dict[col]) in ["integer", "float"] else False
                }
                for col in df.columns
            }
        }
    }

    return mapping

In [3]:
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')

con = mysql_connection_whole_corp()
cursor = con.cursor()

# cursor.execute('select * from whole_corp limit 100')
# result = cursor.fetchall()
# df = pd.DataFrame(result, columns= cursor.column_names)


In [ ]:
con.close()

In [4]:
cursor.execute('select * from wholecorp_clusters_vector')
df_vector = pd.DataFrame(cursor.fetchall(), columns=cursor.column_names)

# df_vector.drop_duplicates(subset='統一編號',inplace=True)

len(df_vector)


2759468

In [7]:
df_vector.drop_duplicates(subset='統一編號',inplace=True)

len(df_vector)

1379734

In [16]:
cursor.execute('select * from wholecorp_clusters_vector_1')
df_vector_1 = pd.DataFrame(cursor.fetchall(), columns=cursor.column_names)

df_vector_1.drop_duplicates(subset='統一編號',inplace=True)

In [17]:
len(df_vector_1)

1379734

In [12]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib, os, csv

user = "root"
password = "!QAZ2wsx"
host = "localhost"
port = 3307
db   = "whole_corp"
tbl  = "wholecorp_clusters_vector_1"

encoded_pw = urllib.parse.quote_plus(password)
# IMPORTANT: allow_local_infile=1 for mysqlconnector (or local_infile=1 for PyMySQL)
db_url = f"mysql+mysqlconnector://{user}:{encoded_pw}@{host}:{port}/{db}?allow_local_infile=1"

engine = create_engine(db_url, pool_pre_ping=True)

# with engine.connect() as conn:
#     conn.execute(text("""
#         CREATE TABLE IF NOT EXISTS whole_corp (
#             統一編號 CHAR(8) NOT NULL,
#             公司名稱 VARCHAR(50),
#             負責人 VARCHAR(20),
#             登記地址 VARCHAR(200),
#             資本額 VARCHAR(50),
#             營業項目及代碼表 VARCHAR(200),
#             縣市名稱 VARCHAR(20),
#             區域名稱 VARCHAR(20),
#             縣市區域 VARCHAR(20),
#             類別_全 VARCHAR(200),
#             官網 VARCHAR(100),
#             電話 VARCHAR(20),
#             PRIMARY KEY (統一編號)
#         ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
#     """))
# 1) Ensure table exists with the right schema.
#    If you don't have it yet, create it once with to_sql but with zero rows
#    (or write a CREATE TABLE manually for precise types).
#    Example (create empty table once):
df_vector.head(0).to_sql(tbl, engine, if_exists="append", index=False)

# 2) Write a clean TSV to disk (much safer than CSV for commas/quotes)
tmp_path = "/tmp/whole_corp_load.tsv"  # on Windows: r"C:\temp\whole_corp_load.tsv"
os.makedirs(os.path.dirname(tmp_path), exist_ok=True)

# Write TSV: represent NULLs as \N (MySQL understands \N as NULL if not quoted)
df_vector.to_csv(
    tmp_path,
    sep="\t",
    index=False,
    na_rep="\\N",
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
)

# 3) Bulk load
with engine.begin() as conn:

    # optional but helps: disable checks while loading
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0"))
    conn.execute(text("SET UNIQUE_CHECKS=0"))
    # If you have heavy indexes on the table, consider dropping them before and recreating after.

    # LOAD DATA (LOCAL needs allow_local_infile=1)
    conn.execute(text(f"""
        LOAD DATA LOCAL INFILE :path
        INTO TABLE {tbl}
        FIELDS TERMINATED BY '\t'
        ESCAPED BY '\\\\'
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES
    """), {"path": tmp_path})

    conn.execute(text("SET UNIQUE_CHECKS=1"))
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1"))

# Mapping for each tables

# Create index with mapping in ES

In [ ]:
if __name__ == '__main__':
    es = create_es_connection()
    index_name = 'whole_corp'
    if es.indices.exists(index = index_name):
        if str(input('Index {index_name} already exists. Delete? [y/n]').lower()) == 'y':
            es.indices.delete(index=index_name)
        else:
            print('count:', es.count(index = index_name))

    response = es.indices.create(index=index_name, body=corp_mapping())
    
    # Checking mapping in each index
    print(f"/n=============response, es.indices.get_mapping(index={index_name})=============")
    print(response, es.indices.get_mapping(index=index_name))
    

# Adjusted mapping

In [ ]:
'''-----------------------------Create mapping for user----------------------------------'''
def corp_mapping():
    # Define the settings for the custom analyzer
    body = {
        "properties": {
          "cluster": {
            "type": "keyword"
          },
          "vector": {
            "type": "dense_vector",
            "dims": 100    # // must match your model output
          }
        }
      }

    return body


In [ ]:
es.indices.put_mapping(index='whole_corp', body=corp_mapping())

# Get index mapping

In [ ]:
es.indices.get_mapping(index='whole_corp')

In [ ]:
body = {
    "query":{
        "bool":{
        "must":{
            'exists': { "field": "vector"}
        }
    }
    }
}

In [ ]:
es.search(index='whole_corp', body=body, size=1)

# Import data

In [ ]:
es = create_es_connection()
for index_name in ['whole_corp']:
    try:
        print(f'index name : [{index_name}]',es.count(index=index_name.lower()))
    except Exception as e:
        print(f'\n==============no values in index name [{index_name}]================')
es.close()

In [ ]:
es.indices.delete(index='whole_corp')

# Data import to ES from mysql

In [ ]:
'''=============================================['userESG', 'lifeCircleESG', 'courseESG']==============================================='''
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')


def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    if isinstance(obj, bytearray):
        return obj.decode('utf-8')
    raise TypeError ("Type %s not serializable" % type(obj))
    

# Generator function to fetch data in batches from MySQL
def fetch_data(cursor, batch_size=1000):
    while True:
        rows = cursor.fetchmany(batch_size)
        if not rows:
            break
        # Convert each row to JSON serializable format
        cleaned_rows = []
        for row in rows:
            cleaned_row = {k: json_serial(v) if isinstance(v, (datetime.datetime, bytearray)) else 0.0 if (k == '資本額') and (v == '\\N') else v for k, v in row.items()}
            cleaned_rows.append(cleaned_row)
        yield cleaned_rows
    

def elastic_import(es, actions):
    response, errors  = helpers.bulk(es, actions, raise_on_error=False)
    if errors:
        print(f'==================================errors: ===============================\n{errors}')
        
    
# Main function to control the flow of the script
def main(table_name, database = None):
    # Connect to MySQL
    
    mysql_conn = mysql_connection_whole_corp() 
        
    cursor = mysql_conn.cursor(dictionary=True)
    
    # Define the SQL query and execute it
    sql_query = f"SELECT * FROM {table_name}"
    cursor.execute(sql_query)
    
    # Connect to Elasticsearch
    es = create_es_connection()

    # Define the Elasticsearch index where the data will be stored
    es_index = table_name.lower()
    
    # Fetch data from MySQL and prepare it for Elasticsearch
#     if es.indices.exists(index = table_name):
#         return 'the index already exists'
    
    for batch in fetch_data(cursor):
        actions = [
            {
                "_index": es_index,
                "_source": row,
            } for row in batch
        ]
        
        # Bulk index the data in Elasticsearch
        elastic_import(es, actions)

    # Cleanup
    cursor.close()
    mysql_conn.close()
    return actions
    
    
if __name__ == "__main__":
    
    #Define paramenter
    database = 'whole_corp'
    table_name = 'whole_corp'
    es_data = main(table_name,database)


# Update index with specific field

In [ ]:
from elasticsearch import helpers
import numpy as np
import pandas as pd
import datetime, boto3
import es_mapping
from connection import ElasticSearchConnectionManager
import sys

sys.path.append(r'C:\Users\mark.ding\big-data-ai-integration-platform\common')
from logger import initlog

logger = initlog(__name__)

# JSON serializer for datetime and bytearray objects
def json_serial(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    if isinstance(obj, bytearray):
        return obj.decode('utf-8')
    raise TypeError(f"Type {type(obj)} not serializable")

# Fetch data from MySQL in batches
def fetch_data(cursor, batch_size=1000):
    while True:
        rows = cursor.fetchmany(batch_size)
        if not rows:
            break
        cleaned_rows = [
            {k: json_serial(v) if isinstance(v, (datetime.datetime, bytearray)) else v for k, v in row.items()}
            for row in rows
        ]
        logger.debug(f"Fetched {len(cleaned_rows)} rows from MySQL.")
        yield cleaned_rows

# Search and update document in Elasticsearch
def search_and_update_field(es, index, source_id, vector_val, cluster_val):
    try:
        search_query = {"query": {"term": {"統一編號": source_id}}}
        search_response = es.search(index=index, body=search_query)
        hits = search_response['hits']['hits']
        
        if not hits:
            logger.info(f"Document with source ID {source_id} does not exist.")
            return False

        for hit in hits:
            doc_id = hit['_id']
            response = es.update(index=index, id=doc_id, body={"doc": {
                "vector": vector_val,
                "cluster" :cluster_val
            }})
            logger.debug(f"Document with source ID {source_id} updated: {response}")
            return True

    except Exception as e:
        logger.error(f"Error updating document with source ID {source_id}: {e}")
        return False

# Bulk import data to Elasticsearch
def elastic_import(es, actions):
    try:
        response, errors = helpers.bulk(es, actions, raise_on_error=False)
        logger.info(f"Imported {response} documents to Elasticsearch, errors: {errors}")
    except Exception as e:
        logger.error(f"Bulk import errors: {e}")

# Update data in Elasticsearch
def update_data_to_es(es, cursor, es_index, table_name):
    updated_count = 0
    create_data_count = 0
    actions = []

    for batch in fetch_data(cursor):
        logger.debug(f"Processing batch with {len(batch)} records.")
        
        df = pd.DataFrame(batch)
        df = df.where(pd.notnull(df), None)  # Replace NaN with None
        df = df.replace({np.nan: None})  # Replace NaN with None

        batch = df.to_dict(orient='records')

        for row in batch:
            source_id = row['統一編號']
            # logger.info(f"Updating document with source ID {source_id}...")
            try:
                print('row:========', row)
                doc_exists = search_and_update_field(es, es_index, source_id, row.get('vector'), row.get('cluster'))
                if not doc_exists:
                    actions.append({"_index": es_index, "_source": row})
                    create_data_count += 1
                    logger.debug(f"Document with source ID {source_id} created.")
                else:
                    updated_count += 1
                    logger.debug(f"Document with source ID {source_id} updated.")

            except Exception as e:
                logger.error(f"Error updating document with source ID {source_id}: {e}")

        if actions:
            logger.info(f"Importing {len(actions)} documents to Elasticsearch index: [{es_index}]...]")
            elastic_import(es, actions)
            actions = []  # Clear actions after bulk import

    logger.info(f"[{table_name}] total documents updated: {updated_count}, and created: {create_data_count}")
    return updated_count, create_data_count

# Main function to handle the ETL process
def etl_process(table_name, es, es_index):
    logger.info(f"Starting ETL process for table: {table_name}.")
    
    # Create MySQL connection
    mysql_conn = ElasticSearchConnectionManager.mysql_connection_whole_corp() 
    cursor = mysql_conn.cursor(dictionary=True)

    date_now_python = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    date_now_sql_query = "SELECT now()"
    cursor.execute(date_now_sql_query)
    date_now_sql = cursor.fetchone()
    date_now_sql = date_now_sql['now()'].strftime("%Y-%m-%d %H:%M:%S")

    # Check if date time is the same in Python and SQL
    if date_now_python != date_now_sql:
        logger.info("Date time is not the same: %s and %s", date_now_python, date_now_sql)
        cursor.execute("SET time_zone = 'Asia/Taipei';")
    else:
        logger.info("Date time is the same: %s", date_now_python)

    # Query to fetch updated records
    sql_query_updatedAt = f"SELECT * FROM {table_name}"
    cursor.execute(sql_query_updatedAt)

    # Update data in Elasticsearch
    update_data_count, create_data_count = update_data_to_es(es, cursor, es_index, table_name)

    mysql_conn.close()
    logger.info(f"ETL process for table: {table_name} completed with {update_data_count} updates and {create_data_count} creations.")
    return update_data_count, create_data_count


def create_index_if_not_exists(es, index_name):
    try:
        es.indices.get_mapping(index=index_name)
        logger.info(f"Index {index_name} already exists.")
    except Exception as e:
        logger.warning(f"Index {index_name} does not exist. Creating new index.")
        mapping = es_mapping.corp_mapping()
        if mapping:
            es.indices.create(index=index_name, body=mapping)
            logger.info(f"Index {index_name} created with the specified mapping.")
        else:
            logger.error(f"Failed to create index {index_name}. No valid mapping found.")


def main():
    logger.info("ETL process started.")
    es = ElasticSearchConnectionManager.get_instance()
    
    table_name = 'wholecorp_clusters_vector'
    
    index_name = table_name.lower()
    
    create_index_if_not_exists(es, index_name)
    
    update_data_count, create_data_count = etl_process(table_name, es, index_name)
    
    logger.info(f"Data update for {table_name} completed, with update_data_count: {update_data_count}, create_data_count: {create_data_count}")
    
    logger.info("ETL process finished.")


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.error(f"ETL process failed due to: {str(e)}", exc_info=True)
